### Find Not Discounted in Allowed List DMI

In [10]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from os.path import join, normpath
import os
import roamability as rb
import re
from collections import namedtuple

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Thu_Dec_26_2019.csv'
result_file = 'RESULT.csv'

# Set the accounts names
sponsor_object = namedtuple('Sponsor', 'sponsor ponsor_id al_name al_pattern')

s1_sponsor = sponsor_object('S1', 1, 'Partner_All', r'[Pp][TAa][rtner]?[\s\w-]*')
s2_sponsor = sponsor_object('S2', 2, 'P4_All', r'[Pp]4[\s\w-]*')
s4_sponsor = sponsor_object('S4', 4, 'MB_ALL', r'[Mm][Bb][\s\w-]*')
s5_sponsor = sponsor_object('S5', 5, 'SMART', r'SMART[\s\w-]*')
s8_sponsor = sponsor_object('S8', 8, 'S8_ALL', r'S8[\s\w-]*')
                
sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s8_sponsor]

#######################################################################################

#######################################################################################

def prepare_toc_to_delete_al(df, downloads, result_file_name):
    """df[['Sponsor','TADIG']], where Sponsor - Allowed list name like Partner_Combined, P4_Combined..."""
    ouf = open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"dataKb":"1","sponsor":"'
    t2='","mo":"1","data":"1","mt":"1","sms":"1","plmnCode":"'
    t3='","moSec":"1"},"type":"remove","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig] in enumerate(df_temp.values):
        ouf.write(t1 + sponsor_name + t2 + tadig + t3)
        if i < num_val:
            ouf.write(',')
    ouf.write(']')
    ouf.close()

In [14]:
# Check not found Allowed Lists
print('Not found Allowed Lists:\n')
for sponsor in df_al.SPONSOR.unique().tolist():
    if sponsor not in [sponsor.sponsor for sponsor in sponsors]:
        print(sponsor)

Not found Allowed Lists:

Sure_RB_Test
S6_ALL
S5_TEST_COS4
S5_TOT_COS4
S5_DRIM01_ALL
S7


In [11]:
# Получение данных TADIG и прайсов с флагом is_discounted = 1

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE AS TADIG,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
###################################################
# Adjustments
df_tadig.drop(
    df_tadig[(df_tadig.TADIG=='YUGPM') & (df_tadig.NETWORK_ID==623)].index, inplace=True)
# c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\Partner_TURTK_YUGPM_not_discounted.msg
###################################################
    
sql_srt=\
'''
SELECT r.NETWORK_ID
,r.SPONSOR_ID AS SPONSOR
,r.data_rate
FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268,329,343,372)
AND r.end_date IS NULL
AND r.is_discounted = 1
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_disc = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    

display(df_tadig.head(3))

replace_sponsor_id_dict = dict((sponsor.ponsor_id, sponsor.sponsor) for sponsor in sponsors)
df_disc.SPONSOR = df_disc.SPONSOR.replace(replace_sponsor_id_dict)
display(df_disc.head(3))

,COUNTRY_NAME,NETWORK_NAME,TADIG,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4
1,Afghanistan,Areeba/MTN,AFGAR,5
2,Afghanistan,Etisalat,AFG55,6


,NETWORK_ID,SPONSOR,data_rate
0,365,S1,0.022
1,1208,S1,0.020
2,164,S4,0.005


In [12]:
# Create df for existing "All" allowed lists and new "Allowed list"

# Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

replace_dict = dict((sponsor.al_pattern, sponsor.sponsor) for sponsor in sponsors)

df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace=replace_dict, regex=True)

df_al = pd.merge(df_al, df_tadig, how='left', on='TADIG')

display(df_al.head(3))

,Sponsor,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,NETWORK_ID
0,Partner,ALBVF,S1,Albania,Vodafone,12
1,Partner_All,GTMTG,S1,Guatemala,Telef nica M viles Guatemala S A,366
2,Partner,ARM01,S1,Armenia,ArmenTel,30


In [13]:
df_temp = pd.merge(df_al, df_disc, how='left', on=['NETWORK_ID','SPONSOR'])
df_temp[df_temp.data_rate.isnull()].to_csv(join(downloads, result_file), index=False)
df_temp[df_temp.data_rate.isnull()]

,Sponsor,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,NETWORK_ID,data_rate
1,Partner_All,GTMTG,S1,Guatemala,Telef nica M viles Guatemala S A,366,NaN
6,P4_STI_Comb,PAKTP,S2,Pakistan,Telenor Pakistan Private Limited,700,NaN
13,P4,PAKTP,S2,Pakistan,Telenor Pakistan Private Limited,700,NaN
30,Partner,CZEET,S1,Czech Republic,O2 Czech Republic a s,234,NaN
41,Partner_Clay_Com,GTMTG,S1,Guatemala,Telef nica M viles Guatemala S A,366,NaN
54,Partner,HKGM3,S1,Hong Kong (China),CSL-M3,393,NaN
55,P4,EGYEM,S2,Egypt,ETISALAT,262,NaN
60,Partner_Com_JWM,GTMTG,S1,Guatemala,Telef nica M viles Guatemala S A,366,NaN
88,Partner_CombNext,GTMTG,S1,Guatemala,Telef nica M viles Guatemala S A,366,NaN
117,Partner,SVKO2,S1,Slovakia,O2 Slovakia s r o,823,NaN


In [ ]:
# Prepare commands to adjust the existing AL of customer by deleting the extras

delete_file_name = 'al_adjust_delete.rcjson' # file name with TOC comands for deleting the extras
df_temp = df_make_changes_custom_current[['Sponsor_CURRENT','TADIG']].dropna().drop_duplicates()
prepare_toc_to_delete_al(df_temp, downloads, delete_file_name)